Paso 1: Capturar Imágenes de un Patrón de Calibración

In [4]:
import cv2 as cv  # OpenCV: Librería para procesamiento de imágenes y visión por computadora
import numpy as np  # NumPy: Librería para operaciones numéricas y manejo de matrices
import glob  # Glob: Librería para encontrar todos los nombres de archivos que coinciden con un patrón específico

# Definir el tamaño del tablero de ajedrez (número de esquinas interiores)
chessboard_size = (3, 3)

# Preparar los puntos del objeto en el espacio 3D (0,0,0), (1,0,0), (2,0,0), ..., (2,2,0)
objp = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)

# Arrays para almacenar los puntos del objeto (3D) y los puntos de la imagen (2D) de todas las imágenes
objpoints = []  # Puntos 3D en el espacio del mundo real
imgpoints = []  # Puntos 2D en el plano de la imagen

# Leer las imágenes de calibración que coinciden con el patrón 'Imagen*.jpg'
images = glob.glob('Imagen*.jpg')

# Iterar sobre cada imagen de calibración
for fname in images:
    # Leer la imagen
    img = cv.imread(fname)
    # Convertir la imagen a escala de grises
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Encontrar las esquinas del tablero de ajedrez en la imagen en escala de grises
    ret, corners = cv.findChessboardCorners(gray, chessboard_size, None)

    # Si se encuentran las esquinas, añadir los puntos del objeto y de la imagen
    if ret:
        # Añadir los puntos del objeto (3D)
        objpoints.append(objp)
        # Añadir los puntos de la imagen (2D)
        imgpoints.append(corners)

        # Dibujar y mostrar las esquinas del tablero de ajedrez en la imagen
        cv.drawChessboardCorners(img, chessboard_size, corners, ret)
        cv.imshow('Esquinas del Tablero de Ajedrez', img)
        cv.waitKey(500)  # Esperar 500 ms antes de mostrar la siguiente imagen

# Cerrar todas las ventanas de OpenCV
cv.destroyAllWindows()

# Calibrar la cámara usando los puntos del objeto y los puntos de la imagen
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# Guardar los parámetros de la cámara (matriz de la cámara y coeficientes de distorsión) en un archivo
np.savez('calibration_data.npz', mtx=mtx, dist=dist)

KeyboardInterrupt: 

: 

Paso 3.1: Aplicar la De-Distorsión a las Imágenes Capturada Estatica

In [ ]:
import cv2 as cv  # OpenCV: Librería para procesamiento de imágenes y visión por computadora
import numpy as np  # NumPy: Librería para operaciones numéricas y manejo de matrices

# Cargar los parámetros de la cámara desde un archivo .npz
with np.load('calibration_data.npz') as data:
    mtx = data['mtx']  # Matriz de la cámara
    dist = data['dist']  # Coeficientes de distorsión

# Leer la imagen estática desde un archivo
static_img = cv.imread('Test1.png')

# Verificar si la imagen se ha cargado correctamente
if static_img is None:
    print("Error: No se pudo cargar la imagen. Verifica la ruta y el nombre del archivo.")
    exit()

# De-distorsionar la imagen estática usando los parámetros de la cámara
h, w = static_img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))
static_img_undistorted = cv.undistort(static_img, mtx, dist, None, newcameramtx)

# Convertir la imagen decolor a escala de grises
gray_static = cv.cvtColor(static_img_undistorted, cv.COLOR_BGR2GRAY)

# Aplicar un desenfoque gaussiano para reducir el ruido en la imagen
blurred_static = cv.GaussianBlur(gray_static, (5, 5), 0)

# Aplicar un umbral binario para detectar los bordes en la imagen
ret, thresh_static = cv.threshold(blurred_static, 127, 255, cv.THRESH_BINARY)

# Encontrar los contornos en la imagen umbralizada
contours_static, _ = cv.findContours(thresh_static, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

# Filtrar los contornos por área mínima
min_area = 100  # Ajusta este valor según sea necesario
filtered_contours_static = [cnt for cnt in contours_static if cv.contourArea(cnt) > min_area]

# Dibujar los contornos filtrados en la imagen decolor
cv.drawContours(static_img_undistorted, filtered_contours_static, -1, (0, 255, 0), 2)

# Mostrar la imagen estática con los contornos dibujados
cv.imshow('Contornos en Imagen Estática', static_img_undistorted)
cv.waitKey(500)  # Esperar 500 ms antes de cerrar la ventana

# Guardar las coordenadas de los contornos de la imagen estática
contour_coords_static = [cnt for cnt in filtered_contours_static]

Paso 3.2: Aplicar la De-Distorsión a las Imágenes Capturadas en Tiempo Real

In [ ]:
import cv2 as cv  # OpenCV: Librería para procesamiento de imágenes y visión por computadora
import cv2.aruco as aruco  # Módulo Aruco de OpenCV: Librería para trabajar con marcadores Aruco

# Definir el diccionario de Aruco y el ID del marcador
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_6X6_250)  # Diccionario de marcadores Aruco predefinido
marker_id = 0  # ID del marcador que se va a generar
marker_size = 700  # Tamaño del marcador en píxeles

# Generar el marcador
marker = aruco.generateImageMarker(aruco_dict, marker_id, marker_size)  # Generar la imagen del marcador Aruco

# Guardar el marcador como una imagen
cv.imwrite('aruco_marker.png', marker)  # Guardar la imagen del marcador en un archivo

# Mostrar el marcador
cv.imshow('Marcador de Aruco', marker)  # Mostrar la imagen del marcador en una ventana
cv.waitKey(0)  # Esperar a que se presione una tecla para cerrar la ventana
cv.destroyAllWindows()  # Cerrar todas las ventanas de OpenCV

Paso 3.3: Ejemplo de detección de Aruco en vivo

In [ ]:
import cv2 as cv  # OpenCV: Librería para procesamiento de imágenes y visión por computadora
import cv2.aruco as aruco  # Módulo Aruco de OpenCV: Librería para trabajar con marcadores Aruco
import numpy as np  # NumPy: Librería para operaciones numéricas y manejo de matrices

# Cargar los parámetros de la cámara desde un archivo .npz
with np.load('calibration_data.npz') as data:
    mtx = data['mtx']  # Matriz de la cámara
    dist = data['dist']  # Coeficientes de distorsión

# Definir el diccionario de Aruco y los parámetros del detector
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_6X6_250)  # Diccionario de marcadores Aruco predefinido
parameters = aruco.DetectorParameters()  # Parámetros del detector de Aruco

# Ajustar los parámetros del detector para mejorar la velocidad y precisión
parameters.adaptiveThreshWinSizeMin = 3
parameters.adaptiveThreshWinSizeMax = 23
parameters.adaptiveThreshWinSizeStep = 10
parameters.cornerRefinementMethod = aruco.CORNER_REFINE_SUBPIX

# Definir el tamaño del marcador en el mundo real (en metros)
marker_length = 0.1  # Ajusta este valor según sea necesario

# Iniciar la captura de video desde la cámara web
cap = cv.VideoCapture(1)

# Verificar si la cámara se abrió correctamente
if not cap.isOpened():
    print("Error: No se pudo abrir la cámara.")
    exit()

# Reducir la resolución del video para aumentar la velocidad de procesamiento
cap.set(cv.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv.CAP_PROP_FRAME_HEIGHT, 480)

while True:
    # Capturar un cuadro del video
    ret, frame = cap.read()
    if not ret:
        print("Error: No se pudo capturar el cuadro.")
        break

    # Convertir el cuadro a escala de grises
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # Detectar los marcadores de Aruco en el cuadro
    corners, ids, rejected = aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

    # Si se detectan marcadores, calcular la posición y orientación de la cámara
    if ids is not None:
        rvecs, tvecs, _ = aruco.estimatePoseSingleMarkers(corners, marker_length, mtx, dist)

        # Dibujar los marcadores y los ejes en el cuadro
        for rvec, tvec in zip(rvecs, tvecs):
            aruco.drawDetectedMarkers(frame, corners, ids)
            cv.drawFrameAxes(frame, mtx, dist, rvec, tvec, 0.1)

            # Proyectar el rectángulo prohibido en la imagen
            # Definir las esquinas del rectángulo en el espacio real
            rect_corners = np.array([
                [-0.5, -0.5, 0],
                [0.5, -0.5, 0],
                [0.5, 0.5, 0],
                [-0.5, 0.5, 0]
            ]) * marker_length

            # Proyectar las esquinas del rectángulo en la imagen
            imgpts, _ = cv.projectPoints(rect_corners, rvec, tvec, mtx, dist)

            # Dibujar el rectángulo en la imagen
            imgpts = np.int32(imgpts).reshape(-1, 2)
            cv.polylines(frame, [imgpts], isClosed=True, color=(0, 0, 255), thickness=2)

    # Mostrar el cuadro con los marcadores y el rectángulo proyectado
    cv.imshow('Detección de Marcadores de Aruco', frame)

    # Salir del bucle si se presiona la tecla 'q'
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar la captura de video y cerrar todas las ventanas
cap.release()
cv.destroyAllWindows()

Paso 4: Deteccion de Personas Implementada

In [ ]:
!pip install tensorflow tensorflow-hub
!pip install pygame

^C


In [1]:
import cv2 as cv  # OpenCV: Librería para procesamiento de imágenes y visión por computadora
import numpy as np  # NumPy: Librería para operaciones numéricas y manejo de matrices
import tensorflow as tf  # TensorFlow: Librería para aprendizaje automático y redes neuronales
import tensorflow_hub as hub  # TensorFlow Hub: Librería para reutilizar modelos de TensorFlow preentrenados
import threading  # Threading: Librería para trabajar con hilos (threads) en Python
import pygame  # Pygame: Librería para desarrollo de videojuegos y reproducción de audio
import os  # OS: Librería para interactuar con el sistema operativo

# Inicializar pygame mixer para reproducir sonidos
pygame.mixer.init()

# Cargar los parámetros de la cámara desde un archivo de calibración
with np.load('calibration_data.npz') as data:
    mtx = data['mtx']  # Matriz de la cámara
    dist = data['dist']  # Coeficientes de distorsión

# Cargar el modelo de detección de personas desde TensorFlow Hub
model = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")  # Modelo SSD MobileNet V2 para detección de objetos


pygame 2.6.0 (SDL 2.28.4, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html



In [2]:
# Cargar la imagen estática para definir la zona prohibida
static_img = cv.imread('Test1.png')
if static_img is None:
    print("Error: No se pudo cargar la imagen. Verifica la ruta y el nombre del archivo.")
    exit()

# Convertir la imagen a escala de grises y aplicar desenfoque gaussiano para reducir el ruido
gray_static = cv.cvtColor(static_img, cv.COLOR_BGR2GRAY)
blurred_static = cv.GaussianBlur(gray_static, (5, 5), 0)

# Aplicar un umbral binario para detectar los bordes en la imagen
ret, thresh_static = cv.threshold(blurred_static, 127, 255, cv.THRESH_BINARY)

# Encontrar los contornos en la imagen umbralizada
contours_static, _ = cv.findContours(thresh_static, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

# Dibujar los contornos en la imagen estática
cv.drawContours(static_img, contours_static, -1, (0, 255, 0), 2)
cv.imshow('Contornos en Imagen Estática', static_img)
cv.waitKey(500)  # Esperar 500 ms antes de cerrar la ventana

# Función para proyectar la región prohibida en la imagen
def project_region(contours_static, h, w):
    projected_contours = []
    for contour in contours_static:
        # Convertir el contorno a un formato adecuado y reescalar a las dimensiones del frame actual
        contour_2d = np.array([[[pt[0][0], pt[0][1]]] for pt in contour], dtype=np.int32)
        scale_x = w / static_img.shape[1]
        scale_y = h / static_img.shape[0]
        contour_2d[:, :, 0] = contour_2d[:, :, 0] * scale_x
        contour_2d[:, :, 1] = contour_2d[:, :, 1] * scale_y
        projected_contours.append(contour_2d)
    return projected_contours

# Función para realizar la detección de objetos
def detect_objects(image):
    # Convertir la imagen a RGB y luego a un tensor de TensorFlow
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    image = tf.convert_to_tensor(image, dtype=tf.uint8)[tf.newaxis, ...]
    result = model(image)  # Realizar la detección de objetos
    return result

# Procesar los cuadros en tiempo real
def process_frame(frame, h, w):
    # Proyectar la región prohibida en el frame actual
    projected_region = project_region(contours_static, h, w)
    result = detect_objects(frame)  # Detectar objetos en el frame
    boxes = result["detection_boxes"].numpy()[0]
    class_ids = result["detection_classes"].numpy()[0].astype(np.int32)
    scores = result["detection_scores"].numpy()[0]

    # Dibujar la región prohibida en la imagen
    for contour in projected_region:
        cv.polylines(frame, [contour], isClosed=True, color=(0, 0, 255), thickness=2)

    # Dibujar las detecciones de personas y verificar si están en la región prohibida
    for box, class_id, score in zip(boxes, class_ids, scores):
        if score < 0.5 or class_id != 1:  # Filtrar detecciones con baja confianza o que no sean personas
            continue
        y1, x1, y2, x2 = box
        x1, x2, y1, y2 = int(x1 * w), int(x2 * w), int(y1 * h), int(y2 * h)
        person_box = np.array([[x1, y1], [x2, y1], [x2, y2], [x1, y2]])
        cv.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Azul para la detección de personas
        label = f"Person, Score: {score:.2f}"
        cv.putText(frame, label, (x1, y1 - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # Verificar si la persona está en la región prohibida
        for contour in projected_region:
            if cv.pointPolygonTest(contour, ((x1 + x2) // 2, (y1 + y2) // 2), False) >= 0:
                color = (0, 0, 255)  # Rojo si está en la región prohibida
                if os.path.exists('beep-warning-6387.mp3'):
                    pygame.mixer.music.load('beep-warning-6387.mp3')
                    pygame.mixer.music.play()
                else:
                    print("Error: El archivo de sonido 'beep-warning-6387.mp3' no se encuentra.")
                break
            else:
                color = (0, 255, 0)  # Verde si está fuera de la región prohibida
        cv.rectangle(frame, (x1, y1), (x2, y2), color, 2)

    return frame

# Iniciar la captura de video desde la cámara web
cap = cv.VideoCapture(1)
if not cap.isOpened():
    print("Error: No se pudo abrir la cámara.")
    exit()

# Configurar la resolución del video
cap.set(cv.CAP_PROP_FRAME_WIDTH, 320)
cap.set(cv.CAP_PROP_FRAME_HEIGHT, 240)

frame = None
processed_frame = None  # Inicializar la variable processed_frame
lock = threading.Lock()  # Crear un bloqueo para sincronizar el acceso a los cuadros

# Función para capturar cuadros de video en un hilo separado
def capture_frames():
    global frame
    while True:
        ret, frame = cap.read()
        if not ret:
            break

# Iniciar el hilo de captura de cuadros
capture_thread = threading.Thread(target=capture_frames)
capture_thread.start()

# Procesar y mostrar los cuadros en tiempo real
while True:
    if frame is not None:
        h, w = frame.shape[:2]
        with lock:
            processed_frame = process_frame(frame.copy(), h, w)

    if processed_frame is not None:
        cv.imshow('Detección de Objetos y Zona Prohibida', processed_frame)

    if cv.waitKey(1) & 0xFF == ord('q'):  # Salir del bucle si se presiona la tecla 'q'
        break

# Liberar la captura de video y cerrar todas las ventanas
cap.release()
cv.destroyAllWindows()

pygame 2.6.0 (SDL 2.28.4, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
OpenCV version: 4.10.0
TensorFlow version: 2.17.0
Pygame version: 2.6.0

In [ ]:
import cv2 as cv
import numpy as np
import tensorflow as tf
import threading
import os
import pygame

# Inicializar pygame para reproducir sonido
pygame.mixer.init()

# Cargar la imagen de referencia para la zona prohibida
reference_image = cv.imread('Test1.png')
if reference_image is None:
    print("Error: No se pudo cargar la imagen de referencia.")
    exit()

# Definir la zona prohibida basada en la imagen de referencia
def define_prohibited_zone(image):
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    blurred = cv.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv.threshold(blurred, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
    contours, _ = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    if contours:
        largest_contour = max(contours, key=cv.contourArea)
        epsilon = 0.02 * cv.arcLength(largest_contour, True)
        approx_contour = cv.approxPolyDP(largest_contour, epsilon, True)
        return approx_contour
    return None

prohibited_zone = define_prohibited_zone(reference_image)

# Función para realizar la detección de objetos
def detect_objects(image):
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    image = tf.convert_to_tensor(image, dtype=tf.uint8)[tf.newaxis, ...]
    result = model(image)
    return result

# Función para detectar el objeto de referencia en el cuadro en vivo
def detect_reference_object(frame, reference_image):
    sift = cv.SIFT_create()
    kp1, des1 = sift.detectAndCompute(reference_image, None)
    kp2, des2 = sift.detectAndCompute(frame, None)
    
    if des1 is None or des2 is None:
        return None
    
    bf = cv.BFMatcher(cv.NORM_L2, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)
    if len(matches) > 10:
        src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
        M, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC, 5.0)
        h, w = reference_image.shape[:2]
        pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
        dst = cv.perspectiveTransform(pts, M)
        return dst
    return None

# Función modificada para procesar los cuadros en tiempo real
def process_frame(frame, mtx, dist):
    # Undistort the frame
    h, w = frame.shape[:2]
    newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))
    undistorted_frame = cv.undistort(frame, mtx, dist, None, newcameramtx)

    # Detect reference object
    reference_object = detect_reference_object(undistorted_frame, reference_image)

    if reference_object is not None:
        # Draw prohibited area
        cv.drawContours(undistorted_frame, [prohibited_zone], 0, (0, 0, 255), 2)

        # Detect objects
        result = detect_objects(undistorted_frame)
        boxes = result["detection_boxes"].numpy()[0]
        class_ids = result["detection_classes"].numpy()[0].astype(np.int32)
        scores = result["detection_scores"].numpy()[0]

        person_in_prohibited_area = False

        for box, class_id, score in zip(boxes, class_ids, scores):
            if score < 0.5 or class_id != 1:  # Class 1 is person in COCO dataset
                continue
            y1, x1, y2, x2 = box
            x1, x2, y1, y2 = int(x1 * w), int(x2 * w), int(y1 * h), int(y2 * h)

            # Check if person is in prohibited area
            person_bottom_center = ((x1 + x2) // 2, y2)
            if cv.pointPolygonTest(prohibited_zone, person_bottom_center, False) >= 0:
                color = (0, 0, 255)  # Red if in prohibited area
                person_in_prohibited_area = True
            else:
                color = (0, 255, 0)  # Green if outside prohibited area

            cv.rectangle(undistorted_frame, (x1, y1), (x2, y2), color, 2)
            label = f"Person, Score: {score:.2f}"
            cv.putText(undistorted_frame, label, (x1, y1 - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        if person_in_prohibited_area:
            cv.putText(undistorted_frame, "ALARMA: Intruso en zona prohibida", (10, 30), cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            if os.path.exists('beep-warning-6387.mp3'):
                pygame.mixer.music.load('beep-warning-6387.mp3')
                pygame.mixer.music.play()
            else:
                print("Error: El archivo de sonido 'beep-warning-6387.mp3' no se encuentra.")

    return undistorted_frame

# Iniciar la captura de video desde la cámara web
cap = cv.VideoCapture(1)
if not cap.isOpened():
    print("Error: No se pudo abrir la cámara.")
    exit()

cap.set(cv.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv.CAP_PROP_FRAME_HEIGHT, 480)

frame = None
processed_frame = None
lock = threading.Lock()

def capture_frames():
    global frame
    while True:
        ret, frame = cap.read()
        if not ret:
            break

capture_thread = threading.Thread(target=capture_frames)
capture_thread.start()

while True:
    if frame is not None:
        with lock:
            processed_frame = process_frame(frame.copy(), mtx, dist)

    if processed_frame is not None:
        cv.imshow('Detección de Objetos y Zona Prohibida', processed_frame)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

In [2]:
import cv2
import tensorflow as tf
import pygame

# Obtener la versión de OpenCV
opencv_version = cv2.__version__
print(f"OpenCV version: {opencv_version}")

# Obtener la versión de TensorFlow
tensorflow_version = tf.__version__
print(f"TensorFlow version: {tensorflow_version}")

# Obtener la versión de Pygame
pygame_version = pygame.__version__
print(f"Pygame version: {pygame_version}")

pygame 2.6.0 (SDL 2.28.4, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
OpenCV version: 4.10.0
TensorFlow version: 2.17.0
Pygame version: 2.6.0
